In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import qiskit as qk
qk.IBMQ.load_account()

from system import Model,Pairing,Heisenberg
from algorithms import PhaseEstimation

In [2]:
def binaryfrac(bitstr):
    val = 0
    for i,bit in enumerate(bitstr):
        val += int(bit)*(2**(-i-1))
    return val

### Define variables and model class

In [3]:
dt = 0.005
steps = 100
t = steps*dt

h0 = 1
delta = 1
g = 1
Emax = 2
sim = 4
work = 6
ancilla = 1
#model = Pairing(work,sim,ancilla,Emax,delta,g)
model = Pairing(6,4,1,2,1,1)
#model = Heisenberg(8,2,0,4,1)

                    
 q0_0: |0>──────────
                    
 q0_1: |0>──────────
                    
 q0_2: |0>──────────
                    
 q0_3: |0>──────────
                    
 q0_4: |0>──────────
                    
 q0_5: |0>──────────
          ┌───┐     
 q0_6: |0>┤ H ├──■──
          └───┘┌─┴─┐
 q0_7: |0>─────┤ X ├
          ┌───┐└───┘
 q0_8: |0>┤ H ├──■──
          └───┘┌─┴─┐
 q0_9: |0>─────┤ X ├
               └───┘
q0_10: |0>──────────
                    
  c0_0: 0 ══════════
                    
  c0_1: 0 ══════════
                    
  c0_2: 0 ══════════
                    
  c0_3: 0 ══════════
                    
  c0_4: 0 ══════════
                    
  c0_5: 0 ══════════
                    
  c0_6: 0 ══════════
                    
  c0_7: 0 ══════════
                    
  c0_8: 0 ══════════
                    
  c0_9: 0 ══════════
                    
 c0_10: 0 ══════════
                    


## Do phase estimation

In [4]:
model = PhaseEstimation(model,t=t,dt=dt)

KeyboardInterrupt: 

In [ ]:
# Measurement
model = model.measure(t)

### Plot results

In [ ]:
x = model.plotX
y = model.plotY
plt.plot(x,y)
plt.show()

#### Alternative measurement plot

In [ ]:
res = model.result
measurements = []
for key,value in res.items():
    key_ = key[sim:]
    eigenstate = key[1:(sim)]
    eigenstate = eigenstate[::-1]
    decimal = 0
    for i,bit in enumerate(key_):
        decimal += int(bit)*2**(-i-1)
    if value != 0:
        measurements.append(np.array([eigenstate, model.Emax-decimal*2*np.pi/t, value]))

measurements = np.array(measurements)
x = measurements[:,1]
y = measurements[:,2]
idx = np.argsort(x)
x = x[idx]
y = y[idx]
eigdict = {}
for xi in x:
    eigdict[xi] = 0
for xi, yi in zip(x,y):
    eigdict[xi] += int(yi)

x = np.array(list(eigdict.keys())).astype(np.float)
idx = np.argsort(x)
y = np.array(list(eigdict.values())).astype(np.int)
x = x[idx]
y = y[idx]
plt.plot(x,y)
plt.show()

#### Model measurment (copy)

In [ ]:
x = [] # phase
y = [] # hits
for key,val in model.result.items():
    eigenstate = key[1:model.s+1]
    phi = key[model.N-model.w:]
    phi = phi[::-1]
    x.append(model.Emax - (2*np.pi*binaryfrac(phi))/t)
    y.append(val)
x = np.array(x)
y = np.array(y)
idx = np.argsort(x)
x = x[idx]
y = y[idx]
print(x.shape)
# Check if same phase measured for different eigenstates
x_ = []
y_ = []
for i,xi in enumerate(x):
    if i > 0:
        if xi == x_[-1]:
            y_[-1] += y[i]
        else:
            x_.append(xi)
            y_.append(y[i])
    else:
        x_.append(xi)
        y_.append(y[i])
print(len(x_))
plt.plot(x_,y_)
plt.show()

In [ ]:
w = model.w
s = model.s
anc = model.N-1
res = model.result
t = 0.5
i = 0
for key,val in res.items():
    qWork = key[model.N-model.w:]
    qWork = qWork[::-1]
    qSim  = key[1:s+1]
    qAnc  = key[0]
    phase = binaryfrac(qWork)
    if val > 10:
        print('Measurement: {} ({}) -- {}'.format(i,val,key))
        print('  - Eigenstate: {} -> ancilla: {}'.format(qSim,qAnc))
        print('  - Work qubits: {} -> Phase: {} => {}'.format(qWork,phase,2-2*np.pi*phase/t))
        print('\n')
    i += 1